
bbbb

$$
\begin{align}
x(s) &= (l_0 + \Delta x) s \\\\
y(s) &= \frac{a}{3}(l_0 s)^3 + \frac{b}{2}(l_0 s)^2 \\\\
z(s) &= -\frac{c}{3}(l_0 s)^3 - \frac{d}{2}(l_0 s)^2 \\\\
\theta_x(s) &= \Delta \theta_x s \\\\
\theta_y(s) &= c(l_0 s)^2 + d(l_0 s) \\\\
\theta_z(s) &= a(l_0 s)^2 + b(l_0 s) \\\\
(0 \leq &s \leq 1) \\\\
\end{align}
$$

cccc

$$
\phi
$$


In [9]:
import numpy as np  # The next command is required: "pip install -U numpy"
import quaternion   # The next command is required: "pip install numpy-quaternion"


In [10]:
def Beam_Matrix(l0):
    l0_1 = 1 / l0
    l0_2 = 1 / l0 / l0
    l0_3 = 1 / l0 / l0 / l0
    
    ab_inv0 = np.array([[-6 * l0_3, 3 * l0_2], [ 6 * l0_2, -2 * l0_1]])
    cd_inv0 = np.array([[ 6 * l0_3, 3 * l0_2], [-6 * l0_2, -2 * l0_1]])

    return ab_inv0, cd_inv0


def Beam_Coefficient(x0, q0, x1, q1, l0, ab_inv0, cd_inv0):
    q0_inv = q0.inverse()
    x1_ = quaternion.rotate_vectors(q0_inv, x1 - x0)

    dx = x1_ - np.array([l0, 0, 0])
    dq = q0_inv * q1 * 2
    
    ab0 = ab_inv0 @ np.array([dx[1], dq.z])
    cd0 = cd_inv0 @ np.array([dx[2], dq.y])
    
    dx0  = l0 + dx[0]
    dqx0 = dq.x
    
    return ab0, cd0, dx0, dqx0

# 
def Beam_Displacement(x0, q0, l0, s, ab0, cd0, dx0, dqx0):
    
    ls  = l0 * s
    ls2 = ls * ls
    ls3 = ls2 * ls
    
    x_ = dx0 * s
    y_ = ab0[0] / 3 * ls3 + ab0[1] / 2 * ls2
    z_ =-cd0[0] / 3 * ls3 - cd0[1] / 2 * ls2
    
    tx_ = dqx0 * s
    ty_ = cd0[0] * ls2 + cd0[1] * ls
    tz_ = ab0[0] * ls2 + ab0[1] * ls
    
    xs = x0 + quaternion.rotate_vectors(q0, np.array([x_,y_,z_]))
    qs = q0 * quaternion.from_float_array([1, tx_/2, ty_/2, tz_/2]).normalized()
    
    return xs, qs


In [11]:
r0 = 0.1
l0 = 1.0
E0 = 2e11
G0 = 8e10

A0 = np.pi * r0**2
I0 = np.pi / 4 * r0**4
J0 = np.pi / 2 * r0**4

kxx =  1 * E0 * A0 / l0
kyy = 12 * E0 * I0 / l0**3
kyt =  6 * E0 * I0 / l0**2
ktx =  1 * G0 * J0 / l0
kty =  2 * E0 * I0

x0 = np.array([0.0, 0.0, 0.0])
q0 = quaternion.from_float_array([1,0,0,0])
q1 = quaternion.from_float_array([1, 0, 0, 3.17e-05 / 2]).normalized()
x_unit = np.array([1, 0, 0])

x1 = x0 + l0 * (quaternion.rotate_vectors(q0, x_unit)) + np.array([0, 0.0212e-3, 0])

ab_inv0, cd_inv0 = Beam_Matrix(l0)
ab0, cd0, dx0, dqx0 = Beam_Coefficient(x0, q0, x1, q1, l0, ab_inv0, cd_inv0)
xs, qs = Beam_Displacement(x0, q0, l0, 0.0, ab0, cd0, dx0, dqx0)

print(xs, qs)
print(x0, q0)

xs, qs = Beam_Displacement(x0, q0, l0, 1.0, ab0, cd0, dx0, dqx0)

print(xs, qs)
print(x1, q1)


[0. 0. 0.] quaternion(1, 0, 0, 0)
[0. 0. 0.] quaternion(1, 0, 0, 0)
[1.00e+00 2.12e-05 0.00e+00] quaternion(0.999999999874389, 0, 0, 1.58499999960181e-05)
[1.00e+00 2.12e-05 0.00e+00] quaternion(0.999999999874389, 0, 0, 1.58499999980091e-05)


In [12]:
ab1, cd1, dx1, dqx1 = Beam_Coefficient(x1, q1, x0, q0, l0, ab_inv0, cd_inv0)
xs, qs = Beam_Displacement(x1, q1, l0, 0.0, ab1, cd1, dx1, dqx1)

print(xs, qs)
print(x1, q1)

xs, qs = Beam_Displacement(x1, q1, l0, 1.0, ab1, cd1, dx1, dqx1)

print(xs, qs)
print(x0, q0)


[1.00e+00 2.12e-05 0.00e+00] quaternion(0.999999999874389, 0, 0, 1.58499999980091e-05)
[1.00e+00 2.12e-05 0.00e+00] quaternion(0.999999999874389, 0, 0, 1.58499999980091e-05)
[ 0.00000000e+00 -6.77626358e-21  0.00000000e+00] quaternion(1, 0, 0, 1.99092722559871e-15)
[0. 0. 0.] quaternion(1, 0, 0, 0)


In [13]:
n0 = 100
s_array = np.linspace(0, 1, n0)
x_array = np.zeros([3, n0])
q_array = np.zeros([3, n0])
x_array_= np.zeros([3, n0])

for i0 in range(n0):
    xs, qs = Beam_Displacement(x0, q0, l0, s_array[i0], ab0, cd0, dx0, dqx0)
    x_array[:,i0] = xs
    q_array[:,i0] = quaternion.rotate_vectors(qs, x_unit)
    
    xs, qs = Beam_Displacement(x1, q1, l0, 1 - s_array[i0], ab1, cd1, dx1, dqx1)
    x_array_[:,i0] = xs
    
u_array = x_array[:, 2:] - x_array[:, :-2]
u_array = u_array / np.sqrt(np.square(u_array).sum(0))
